In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import TimeSeriesSplit,cross_val_score,train_test_split, GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
df = pd.read_csv("../data/cleaned/global_projection.csv")
df.mean()

Year           2029.000000
CO2(ppm)        475.948631
global_anom       1.679635
sea_level       197.319300
dtype: float64

In [3]:
df = pd.read_csv("../data/cleaned/global_anomalies.csv")
df['ds'] = df['Year'].apply(lambda x:dt.date(x,12,31))
df.tail(2)

,Year,global_anom,All forcings,Human,Natural,Anthropogenic tropospheric aerosol,Greenhouse gases,Land use,Orbital changes,Ozone,Solar,Volcanic,ds
139,2019,1.19,1.108846,0.996126,0.112721,-0.695457,1.537588,-0.233444,0.030672,0.030672,0.030672,0.046275,2019-12-31
140,2020,1.22,1.134581,1.018415,0.116166,-0.709726,1.571692,-0.237720,0.031610,0.031610,0.031610,0.048304,2020-12-31


In [4]:
#Co2 data and projection
co2_df = pd.read_csv('../data/raw/co2/monthly_in_situ_co2_mlo.csv',
    skiprows=lambda x:x in np.arange(0,54),parse_dates=True)
co2_df_projection = pd.read_csv('../data/raw/co2/projection2100.csv',parse_dates=True)
co2_df1 = co2_df.iloc[2:757,[0,4,7]].copy()

In [5]:
co2_df1 = co2_df1.groupby('Year').mean()
final_co2 = co2_df_projection.merge(co2_df1,on='Year',how='left')
final_co2 = final_co2[final_co2['Year']>= 1958].reset_index(drop=True)
final_co2.head()

,Year,Global CO2 Equivalent Emissions (GtonsCO2/year),CO2(ppm),seasonally_adjustedfit(ppm)
0,1958,20.4768,232.269000,315.229000
1,1959,20.5436,315.981667,315.998333
2,1960,21.1158,316.909167,316.902500
3,1961,21.6520,317.643333,317.628333
4,1962,22.1765,318.454167,318.450000


In [6]:
final_co2[final_co2['Year'].isin(range(2006,2011))]

,Year,Global CO2 Equivalent Emissions (GtonsCO2/year),CO2(ppm),seasonally_adjustedfit(ppm)
48,2006,47.9600,381.807500,381.748333
49,2007,48.9497,383.592500,383.573333
50,2008,49.1355,385.449167,385.436667
51,2009,49.1382,387.360000,387.426667
52,2010,50.7118,389.900833,389.808333


In [7]:
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
model_df = final_co2.dropna()
X = model_df['Global CO2 Equivalent Emissions (GtonsCO2/year)'].to_numpy().reshape(-1,1)
y = model_df['CO2(ppm)'].to_numpy().reshape(-1,1)
X_train,X_test,y_train,y_test = train_test_split(X,y)
model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squared Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

0.8319324650699924
0.7899532131031103
Mean Absolute Error: 8.211005151575264
Mean Squared Error: 324.9159811320359
Root Mean Squared Error: 18.025425962568427


In [8]:
pred = model.predict(final_co2[final_co2['Year']>2021]['Global CO2 Equivalent Emissions (GtonsCO2/year)'].to_numpy().reshape(-1,1))
pred_df = pd.DataFrame({'Year':np.arange(2022,2101),'Predictions':pred.ravel()})
pred_df.head()

,Year,Predictions
0,2022,425.642616
1,2023,428.469879
2,2024,431.296864
3,2025,434.123848
4,2026,436.950832


In [9]:
final_co2.iloc[64:,2] = pred.ravel()
final_co2 = final_co2.iloc[:,:3]
final_co2.head()

,Year,Global CO2 Equivalent Emissions (GtonsCO2/year),CO2(ppm)
0,1958,20.4768,232.269000
1,1959,20.5436,315.981667
2,1960,21.1158,316.909167
3,1961,21.6520,317.643333
4,1962,22.1765,318.454167


In [10]:
from sklearn.linear_model import Lasso
model2 = Lasso(alpha=0.1)
model2.fit(X_train,y_train)
pred2 = model2.predict(X_test)
print(model2.score(X_train,y_train))
print(model2.score(X_test,y_test))
print('Mean Absolute Error:', mean_absolute_error(y_test, pred2))
print('Mean Squared Error:', mean_squared_error(y_test, pred2))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred2)))

0.8319324123431402
0.7898860216811382
Mean Absolute Error: 8.213129800933935
Mean Squared Error: 325.0199178173633
Root Mean Squared Error: 18.028308789716338


In [11]:
#temperature predictions
temp_df = df[['Year','global_anom']]
temp_df = final_co2.merge(temp_df,on='Year',how='left')
model_temp_df = temp_df.dropna()
X = model_temp_df[['Global CO2 Equivalent Emissions (GtonsCO2/year)']]
y = model_temp_df['global_anom'].to_numpy().reshape(-1,1)
X_train,X_test,y_train,y_test = train_test_split(X,y)
model3 = LinearRegression()
model3.fit(X_train,y_train)
pred3 = model3.predict(X_test)
print(model3.score(X_train,y_train))
print(model3.score(X_test,y_test))
print('Mean Absolute Error:', mean_absolute_error(y_test, pred3))
print('Mean Squared Error:', mean_squared_error(y_test, pred3))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred3)))

0.865684667184231
0.9268275759867538
Mean Absolute Error: 0.0967418511253936
Mean Squared Error: 0.010363501878526075
Root Mean Squared Error: 0.10180128623217918


In [12]:
features = temp_df[temp_df['Year']>2020][['Global CO2 Equivalent Emissions (GtonsCO2/year)']]
pred3 = model3.predict(features)
temp_df.iloc[63:,3] = pred3.ravel()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 142
Data columns (total 4 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Year                                             143 non-null    int64  
 1   Global CO2 Equivalent Emissions (GtonsCO2/year)  143 non-null    float64
 2   CO2(ppm)                                         143 non-null    float64
 3   global_anom                                      143 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 5.6 KB


In [13]:
# Save the model
import pickle
filename = 'global_anomalie_pred.sav'
pickle.dump(model3, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [14]:
sea_level_df = pd.read_csv('../data/cleaned/sea_level_global.csv')
final_df = temp_df.merge(sea_level_df,on='Year', how='left')
final_df.head()

,Year,Global CO2 Equivalent Emissions (GtonsCO2/year),CO2(ppm),global_anom,diff,rise_year
0,1958,20.4768,232.269000,0.26,NaN,NaN
1,1959,20.5436,315.981667,0.23,NaN,NaN
2,1960,21.1158,316.909167,0.17,NaN,NaN
3,1961,21.6520,317.643333,0.26,NaN,NaN
4,1962,22.1765,318.454167,0.23,NaN,NaN


In [15]:
#sea_level predictions
model_sea_df = final_df.dropna()
X = model_sea_df[['global_anom','Global CO2 Equivalent Emissions (GtonsCO2/year)']]
y = model_sea_df['diff'].to_numpy().reshape(-1,1)
X_train,X_test,y_train,y_test = train_test_split(X,y)
model3 = LinearRegression()
model3.fit(X_train,y_train)
pred3 = model3.predict(X_test)
print(model3.score(X_train,y_train))
print(model3.score(X_test,y_test))
print('Mean Absolute Error:', mean_absolute_error(y_test, pred3))
print('Mean Squared Error:', mean_squared_error(y_test, pred3))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred3)))

0.9726927412479848
0.9600597569545448
Mean Absolute Error: 5.13852524819119
Mean Squared Error: 37.78257026702597
Root Mean Squared Error: 6.146752822997356


In [16]:
features1 = final_df[final_df['Year']>2020][['global_anom','Global CO2 Equivalent Emissions (GtonsCO2/year)']]
pred3 = model3.predict(features1)
final_df.iloc[63:,4] = pred3.ravel()
final_df = final_df.iloc[:,[0,2,3,4]].rename({'diff':'sea_level'},axis=1)
final_df.tail()

,Year,CO2(ppm),global_anom,sea_level
138,2096,662.623069,3.396050,351.235976
139,2097,663.184160,3.401199,351.851097
140,2098,663.748386,3.406376,352.469654
141,2099,664.309478,3.411525,353.084775
142,2100,664.873704,3.416702,353.703332


In [21]:
#global data projections
final_df.to_csv('../data/cleaned/global_projection.csv',index=False)